In [9]:
import pandas as pd
import time
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

In [10]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [11]:
def getCoinInfo(link):
    driver.get(link)
    count = 1
    rank, name, symbol, market_cap, price,  circulating_supply, volume, change_7d = [], [], [], [], [], [], [], []
    i = 1

    #Load tất cả các trang
    while True:
        try:
            # Chờ nút Load More hiển thị và có thể nhấn
            next_pagination_cmt = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[2]/div[2]/div/div[1]/div[3]/div[2]/button"))
            )
            # Nhấp vào nút
            next_pagination_cmt.click()
            print("Clicked on button next page!")
        except (TimeoutException, NoSuchElementException):
            # Nếu không tìm thấy nút hoặc hết thời gian chờ, kết thúc vòng lặp
            print("No more Load More button found or timeout reached.")
            break
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    
    while True:
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(5)
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        if (screen_height) * i > scroll_height:
            break 

    rank1 =  driver.find_elements(By.XPATH, '//div[@class="cmc-table__table-wrapper-outer"]//table/tbody/tr/td[1]')
    rank =  [s.text for s in rank1] + rank
    name1 =  driver.find_elements(By.XPATH, '//div[@class="cmc-table__table-wrapper-outer"]//table/tbody/tr/td[2]')
    name =  [s.text for s in name1] + name
    symbol1 = driver.find_elements(By.XPATH, '//div[@class="cmc-table__table-wrapper-outer"]//table/tbody/tr/td[3]')
    symbol = [s.text for s in symbol1] + symbol
    market_cap1 = driver.find_elements(By.XPATH, '//div[@class="cmc-table__table-wrapper-outer"]//table/tbody/tr/td[4]')
    market_cap =  [s.text for s in market_cap1] + market_cap
    price1 =  driver.find_elements(By.XPATH, '//div[@class="cmc-table__table-wrapper-outer"]//table/tbody/tr/td[5]')
    price = [s.text for s in price1] + price
    circulating_supply1 =  driver.find_elements(By.XPATH, '//div[@class="cmc-table__table-wrapper-outer"]//table/tbody/tr/td[6]')
    circulating_supply =  [s.text for s in circulating_supply1] + circulating_supply
    volume1 =  driver.find_elements(By.XPATH, '//div[@class="cmc-table__table-wrapper-outer"]//table/tbody/tr/td[7]')
    volume =  [s.text for s in volume1] + volume
    change_7d1 =  driver.find_elements(By.XPATH, '//div[@class="cmc-table__table-wrapper-outer"]//table/tbody/tr/td[10]')
    change_7d =  [s.text for s in change_7d1] + change_7d            
                
    coinmarketcap1 = pd.DataFrame(list(zip(rank, name, symbol, market_cap, price, circulating_supply,   volume,  change_7d)),
                        columns=['rank', 'name', 'symbol', 'market_cap', 'price',  'circulating_supply', 'volume','change_7d']) 
    print("Crawl Complete")
    return coinmarketcap1

url = 'https://coinmarketcap.com/historical/20141130/'
df = getCoinInfo(url)
driver.close()

Clicked on button next page!
Clicked on button next page!
No more Load More button found or timeout reached.
Crawl Complete


In [13]:
# Lấy file CSV
df_unique = df.drop_duplicates(keep='first')
df_unique.to_csv('D://Exam/Coin.csv', index=False)
